In [1]:
pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.8 MB/s eta 0:00:00


In [2]:
pip install uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.5 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image

**Modelin Yüklenmesi**

In [4]:
FaceHairModel = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/FaceHairModel_v1.h5')

In [5]:
def yuzBirlestir(url, url2):
  import cv2
  import dlib
  import numpy
  import urllib
  import base64
  from io import BytesIO
  from PIL import Image

  PREDICTOR_PATH = "/content/drive/MyDrive/ColabNotebooks/shape_predictor_68_face_landmarks.dat"

  DETECTOR = dlib.get_frontal_face_detector()
  PREDICTOR = dlib.shape_predictor(PREDICTOR_PATH)

  def get_landmarks(im):
      rects = DETECTOR(im, 1)
      
      if len(rects) > 1:
          raise Exception("Too many faces")
      if len(rects) == 0:
          raise Exception("Not enough faces")

      return numpy.matrix([[p.x, p.y] for p in PREDICTOR(im, rects[0]).parts()])

  def transformation_from_points(points1, points2):
      points1 = points1.astype(numpy.float64)
      points2 = points2.astype(numpy.float64)

      c1 = numpy.mean(points1, axis=0)
      c2 = numpy.mean(points2, axis=0)
      points1 -= c1
      points2 -= c2

      s1 = numpy.std(points1)
      s2 = numpy.std(points2)
      points1 /= s1
      points2 /= s2

      U, S, Vt = numpy.linalg.svd(points1.T @ points2)
      R = (U @ Vt).T

      return numpy.hstack([(s2 / s1) * R,
                          (c2.T - (s2 / s1) * R @ c1.T)])
      
  def create_mask(points, shape, face_scale):
      groups = [
      # indices of brow and eye landmarks
      [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 36, 37, 38, 39, 40, 41, 42, 43,
      44, 45, 46, 47],
      # indices of mouth and nose landmarks
      [27, 28, 29, 30, 31, 32, 33, 34, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
      58, 59, 60]
      ]
      mask_im = numpy.zeros(shape, dtype=numpy.float64)
      for group in groups:
          cv2.fillConvexPoly(mask_im,
                            cv2.convexHull(body_points[group]),
                            color=(1, 1, 1))
      feather_amount = int(0.2 * face_scale * 0.5) * 2 + 1
      kernel_size = (feather_amount, feather_amount)
      mask_im = (cv2.GaussianBlur (mask_im, kernel_size, 0) > 0) * 1.0 # dilate
      mask_im = cv2.GaussianBlur(mask_im, kernel_size, 0) #blur 
      
      return mask_im

  def correct_colours (warped_face_im, body_im, face_scale):
      blur_amount = int(3 * 0.5 * face_scale) * 2 + 1
      kernel_size = (blur_amount, blur_amount)

      face_im_blur = cv2.GaussianBlur (warped_face_im, kernel_size, 0)
      body_im_blur = cv2.GaussianBlur (body_im, kernel_size, 0)
    
      return numpy.clip(0. + body_im_blur + warped_face_im - face_im_blur, 0, 255)
  
  try:
    req = urllib.request.urlopen(url)
    arr = numpy.asarray(bytearray(req.read()), dtype=numpy.uint8)
    face_im = cv2.imdecode(arr, -1)

    req = urllib.request.urlopen(url2)
    arr = numpy.asarray(bytearray(req.read()), dtype=numpy.uint8)
    body_im = cv2.imdecode(arr, -1)

    face_points = get_landmarks(face_im)
    body_points = get_landmarks(body_im)

    M = transformation_from_points(face_points, body_points)
    warped_face_im = cv2.warpAffine(face_im, M, (body_im.shape[1],body_im.shape[0]))

    face_scale = numpy.std(body_points)
    corrected_face_im = correct_colours(warped_face_im, body_im, face_scale)

    mask_im = create_mask(body_points, body_im.shape, face_scale)
    combined_im = (corrected_face_im * mask_im + body_im * (1 - mask_im))

    cv2.imwrite('/content/drive/MyDrive/ColabNotebooks/combined2.jpg', combined_im)

    with open('/content/drive/MyDrive/ColabNotebooks/combined2.jpg', "rb") as image_file:
        data = base64.b64encode(image_file.read())
  except:
    data = "Hata"

  return data

**API**

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

!pip install Pillow

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
  return {"Hello" : "World"}

@app.get("/sonuclar")
def read_item(url: str = None):
  import urllib
  from io import BytesIO

  header= { 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
            'AppleWebKit/537.11 (KHTML, like Gecko) '
            'Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}

  req = urllib.request.Request(url=url, headers=header)
  page = urllib.request.urlopen(req).read()

  img = image.load_img(BytesIO(page), target_size = (64, 64))
  img = image.img_to_array(img)
  img = np.expand_dims(img , axis = 0)

  result = FaceHairModel.predict(img)

  sekiller = ["Kalp Yüz Şekli", "Uzun Yüz Şekli", "Oval Yüz Şekli", "Yuvarlak Yüz Şekli", "Kare Yüz Şekli"]
  max = result[0][0]
  index = 0

  for i in range(5):
    if max < result[0][i]:
      max = result[0][i]
      index = i

  url2 = ""

  if index == 0:
    url2 = "http://facehair.online/oneri/Kalp/"
  elif index == 1:
    url2 = "http://facehair.online/oneri/Uzun/"
  elif index == 2:
    url2 = "http://facehair.online/oneri/Oval/"
  elif index == 3:
    url2 = "http://facehair.online/oneri/Yuvarlak/"
  elif index == 4:
    url2 = "http://facehair.online/oneri/Kare/"

  resim1 = yuzBirlestir(url, url2+"1.jpg")
  resim2 = yuzBirlestir(url, url2+"2.jpg")
  resim3 = yuzBirlestir(url, url2+"3.jpg")

  return {"Yüz Şekli" : sekiller[index], "birlestirilmis1" : resim1, "birlestirilmis2" : resim2, "birlestirilmis3" : resim3}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install pyngrok
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=649369f11abbd1d7a66050d8a5b1368473791f714a004012e5e95648f47f9de5
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
auth_token = "2Ke6paJeNvLBdBGn6OhjdyZuA3T_rbHcHBnV3AkSqnJe3wT7"
import os
os.system(f"ngrok authtoken {auth_token}")

0

In [9]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)

In [ ]:
!ps aux | grep ngrok

import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

root         771  0.0  0.1 726756 25504 ?        Sl   16:04   0:00 /usr/local/lib/python3.8/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root         781  0.0  0.0   6900  3124 ?        S    16:04   0:00 /bin/bash -c ps aux | grep ngrok
root         783  0.0  0.0   6440   648 ?        S    16:04   0:00 grep ngrok


INFO:     Started server process [175]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


NgrokTunnel: "https://d28b-34-75-206-44.ngrok.io" -> "http://localhost:8000"
1/1 [==============================] - 0s 468ms/step
INFO:     2a02:4780:bad:20:fced:1ff:fe20:232:0 - "GET /sonuclar?url=http://facehair.online/yuklenenler/1677254850Emily-Deschanel.jpg HTTP/1.1" 200 OK


In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)